In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from docx import Document
from docx.shared import Pt

# Start from today
current_date = datetime.today()

# Initialize a variable to store the valid URL
valid_url = None

# Loop until we find a valid page (status code 200)
while True:
    date_str = current_date.strftime("%Y%m%d")
    url = f"https://www.federalreserve.gov/newsevents/pressreleases/monetary{date_str}a.htm"

    try:
        response = requests.get(url)
        response.encoding = 'utf-8'  # Ensure UTF-8 encoding
        soup = BeautifulSoup(response.content, 'html.parser')
        title_element = soup.find(class_='title', string="Federal Reserve issues FOMC statement")
        
        if response.status_code == 200 and title_element:
            valid_url = url
            break
    except:
        pass
    
    current_date -= timedelta(days=1)

# If a valid page is found
if valid_url:
    response = requests.get(valid_url)
    response.raise_for_status()

    soup = BeautifulSoup(response.text, 'html.parser')
    content_div = soup.find('div', id='content', class_='container container__main', role='main')

    if content_div:
        paragraphs = content_div.find_all('p')
        extracted_paragraphs = [p.get_text(strip=True) for p in paragraphs[2:6]]

        # Reverse the order of the paragraphs
        extracted_paragraphs.reverse()

        # Open the original document
        doc = Document('market_analysis.docx')

        # Replace "FOMC_TEXT" with formatted paragraphs
        for i, para in enumerate(doc.paragraphs):
            if "FOMC_TEXT" in para.text:
                # Remove the placeholder paragraph
                parent = para._element.getparent()
                parent.remove(para._element)

                # Insert formatted FOMC paragraphs in reversed order
                for text in extracted_paragraphs:
                    new_para = doc.paragraphs[i].insert_paragraph_before(text)
                    run = new_para.runs[0]
                    run.font.size = Pt(12)

                    # Set paragraph formatting
                    fmt = new_para.paragraph_format
                    fmt.line_spacing = 1.5
                    fmt.space_after = Pt(6)
                break

        # Save the updated document
        doc.save('updated_market_analysis.docx')
    else:
        print("The specified content div was not found.")
else:
    print("No valid FOMC page found.")
